In [0]:
import requests, pandas as pd

In [0]:
raw_csv_url = "https://raw.githubusercontent.com/databricks/LearningSparkV2/master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

In [0]:
flight_delays_raw = pd.read_csv(raw_csv_url)

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
flight_delays_schema = StructType([StructField("date", StringType(), False),
                                   StructField("delay", IntegerType(), False),
                                   StructField("distance", IntegerType(), False),
                                   StructField("origin", StringType(), False),
                                   StructField("destination", StringType(), False)]
                                 )

In [0]:
# Read the pandas dafatframe into a spark DataFrame and save as a Temp View
flight_delays_df = spark.createDataFrame(flight_delays_raw, schema = flight_delays_schema).createOrReplaceTempView('flight_delays')

/databricks/spark/python/pyspark/sql/pandas/conversion.py:340: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Expected a string or bytes dtype, got int64
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

##### 1. Find all flights whose distance is greater than 1,000 miles

In [0]:
display(sql("Select * from flight_delays \
             where distance > 1000"))

##### 2. Find all flights between San Francisco (SFO) and Chicago (ORD) with at least a two-hour delay

In [0]:
display(sql("Select * from flight_delays \
             where ((origin = 'SFO' and destination = 'ORD') OR (origin = 'ORD' and destination = 'SFO')) \
             AND delay >= 2 \
            "))

In [0]:
display(sql("Select * from flight_delays \
             where ((origin || destination = 'SFOORD') OR (origin || destination = 'ORDSFO')) \
             AND delay >= 2 \
            "))

##### 3. Label all US flights, regardless of origin and destination, with an indication of the delays they experienced to a new column flight_delays: 
Very Long Delays (> 6 hours),
Long Delays (2–6 hours), 
Short Delays (1-2 hours),
Tolerable Delays (0-1 hour),
No Delay (0 hour),
Early (< 0 hour)

In [0]:
display(sql("Select date, \
                    delay as delay_in_mins, \
                    distance, \
                    origin, \
                    destination, \
                    CASE \
                        WHEN delay > 360 THEN 'Very Long Delays' \
                        WHEN delay BETWEEN 121 AND 360 THEN 'Long Delays' \
                        WHEN delay BETWEEN 60 AND 120 THEN 'Short Delays' \
                        WHEN delay < 60 AND delay > 1 THEN 'Tolerable Delays' \
                        WHEN delay = 0 THEN 'No Delays' \
                        ELSE 'Early' \
                    END AS delay_type \
            FROM flight_delays \
           "))